In [1]:
import numpy as np 
import pandas as pd 
import datetime
from datetime import datetime
from datetime import datetime, timedelta
from tqdm import tqdm_notebook as tqdm
import calendar
import math
import time

from multiprocessing import Pool, TimeoutError, Lock, Value, current_process, cpu_count
from functools import partial

import RTDFeature_function as func

In [2]:
data = pd.read_csv('D:\Strategic_Trading\PriceForecast\data\RTD\RTD.csv', converters = {"ts": str})
data["ts"] = pd.to_datetime(data["ts"])
data

,ts,open,high,low,close,vol
0,2006-01-02 08:46:00,6451,6490,6411,6478,775
1,2006-01-02 08:47:00,6470,6480,6465,6470,589
2,2006-01-02 08:48:00,6472,6473,6461,6465,633
3,2006-01-02 08:49:00,6464,6467,6456,6457,681
4,2006-01-02 08:50:00,6460,6463,6452,6455,385
...,...,...,...,...,...,...
812095,2016-12-30 13:41:00,9261,9262,9260,9262,442
812096,2016-12-30 13:42:00,9261,9262,9261,9261,275
812097,2016-12-30 13:43:00,9261,9263,9261,9262,672
812098,2016-12-30 13:44:00,9261,9263,9261,9261,646


In [4]:
timelist = data["ts"].dt.time.unique().tolist()
timedict = {}
for i, t in enumerate(timelist): 
    timedict[t] = i

In [5]:
data["IntraDaily_Time"] = data.apply(func.IntraDailyTime, timedict=timedict, axis=1)
data

,ts,open,high,low,close,vol,IntraDaily_Time
0,2006-01-02 08:46:00,6451,6490,6411,6478,775,0.000000e+00
1,2006-01-02 08:47:00,6470,6480,6465,6470,589,1.050681e-02
2,2006-01-02 08:48:00,6472,6473,6461,6465,633,2.101245e-02
3,2006-01-02 08:49:00,6464,6467,6456,6457,681,3.151578e-02
4,2006-01-02 08:50:00,6460,6463,6452,6455,385,4.201562e-02
...,...,...,...,...,...,...,...
812095,2016-12-30 13:41:00,9261,9262,9260,9262,442,4.201562e-02
812096,2016-12-30 13:42:00,9261,9262,9261,9261,275,3.151578e-02
812097,2016-12-30 13:43:00,9261,9263,9261,9262,672,2.101245e-02
812098,2016-12-30 13:44:00,9261,9263,9261,9261,646,1.050681e-02


In [8]:
data["Month"] = data["ts"].dt.month
data

,ts,open,high,low,close,vol,IntraDaily_Time,Month
0,2006-01-02 08:46:00,6451,6490,6411,6478,775,0.000000e+00,1
1,2006-01-02 08:47:00,6470,6480,6465,6470,589,1.050681e-02,1
2,2006-01-02 08:48:00,6472,6473,6461,6465,633,2.101245e-02,1
3,2006-01-02 08:49:00,6464,6467,6456,6457,681,3.151578e-02,1
4,2006-01-02 08:50:00,6460,6463,6452,6455,385,4.201562e-02,1
...,...,...,...,...,...,...,...,...
812095,2016-12-30 13:41:00,9261,9262,9260,9262,442,4.201562e-02,12
812096,2016-12-30 13:42:00,9261,9262,9261,9261,275,3.151578e-02,12
812097,2016-12-30 13:43:00,9261,9263,9261,9262,672,2.101245e-02,12
812098,2016-12-30 13:44:00,9261,9263,9261,9261,646,1.050681e-02,12


In [11]:
data["Weekday"] = data["ts"].dt.weekday + 1
data

,ts,open,high,low,close,vol,IntraDaily_Time,Month,Weekday
0,2006-01-02 08:46:00,6451,6490,6411,6478,775,0.000000e+00,1,1
1,2006-01-02 08:47:00,6470,6480,6465,6470,589,1.050681e-02,1,1
2,2006-01-02 08:48:00,6472,6473,6461,6465,633,2.101245e-02,1,1
3,2006-01-02 08:49:00,6464,6467,6456,6457,681,3.151578e-02,1,1
4,2006-01-02 08:50:00,6460,6463,6452,6455,385,4.201562e-02,1,1
...,...,...,...,...,...,...,...,...,...
812095,2016-12-30 13:41:00,9261,9262,9260,9262,442,4.201562e-02,12,5
812096,2016-12-30 13:42:00,9261,9262,9261,9261,275,3.151578e-02,12,5
812097,2016-12-30 13:43:00,9261,9263,9261,9262,672,2.101245e-02,12,5
812098,2016-12-30 13:44:00,9261,9263,9261,9261,646,1.050681e-02,12,5


In [15]:
year_list = data["ts"].dt.year.unique().tolist()
month_list = data["ts"].dt.month.unique().tolist()
month_dict = {}
for y in year_list:
    month_dict[y] = {}
    for m in month_list:
        d = data[(data["ts"].dt.year == y) & (data["ts"].dt.month == m)]["ts"].dt.date.unique().tolist()
        month_dict[y][m] = {}
        for i, date in enumerate(d):
            month_dict[y][m][date] = i

month_dict

{2006: {1: {datetime.date(2006, 1, 2): 0,
   datetime.date(2006, 1, 3): 1,
   datetime.date(2006, 1, 4): 2,
   datetime.date(2006, 1, 5): 3,
   datetime.date(2006, 1, 6): 4,
   datetime.date(2006, 1, 9): 5,
   datetime.date(2006, 1, 10): 6,
   datetime.date(2006, 1, 11): 7,
   datetime.date(2006, 1, 12): 8,
   datetime.date(2006, 1, 13): 9,
   datetime.date(2006, 1, 16): 10,
   datetime.date(2006, 1, 17): 11,
   datetime.date(2006, 1, 18): 12,
   datetime.date(2006, 1, 19): 13,
   datetime.date(2006, 1, 20): 14,
   datetime.date(2006, 1, 23): 15,
   datetime.date(2006, 1, 24): 16,
   datetime.date(2006, 1, 25): 17},
  2: {datetime.date(2006, 2, 3): 0,
   datetime.date(2006, 2, 6): 1,
   datetime.date(2006, 2, 7): 2,
   datetime.date(2006, 2, 8): 3,
   datetime.date(2006, 2, 9): 4,
   datetime.date(2006, 2, 10): 5,
   datetime.date(2006, 2, 13): 6,
   datetime.date(2006, 2, 14): 7,
   datetime.date(2006, 2, 15): 8,
   datetime.date(2006, 2, 16): 9,
   datetime.date(2006, 2, 17): 10,
   

In [16]:
data["Intramonth"] = data.apply(func.IntraMonth, month_dict=month_dict, axis=1)
data

,ts,open,high,low,close,vol,IntraDaily_Time,Month,Weekday,Intramonth
0,2006-01-02 08:46:00,6451,6490,6411,6478,775,0.000000e+00,1,1,0.0
1,2006-01-02 08:47:00,6470,6480,6465,6470,589,1.050681e-02,1,1,0.0
2,2006-01-02 08:48:00,6472,6473,6461,6465,633,2.101245e-02,1,1,0.0
3,2006-01-02 08:49:00,6464,6467,6456,6457,681,3.151578e-02,1,1,0.0
4,2006-01-02 08:50:00,6460,6463,6452,6455,385,4.201562e-02,1,1,0.0
...,...,...,...,...,...,...,...,...,...,...
812095,2016-12-30 13:41:00,9261,9262,9260,9262,442,4.201562e-02,12,5,1.0
812096,2016-12-30 13:42:00,9261,9262,9261,9261,275,3.151578e-02,12,5,1.0
812097,2016-12-30 13:43:00,9261,9263,9261,9262,672,2.101245e-02,12,5,1.0
812098,2016-12-30 13:44:00,9261,9263,9261,9261,646,1.050681e-02,12,5,1.0


In [17]:
data["IntramonthPeriod"] = data.apply(func.MonthPeriod, axis=1)
data

,ts,open,high,low,close,vol,IntraDaily_Time,Month,Weekday,Intramonth,IntramonthPeriod
0,2006-01-02 08:46:00,6451,6490,6411,6478,775,0.000000e+00,1,1,0.0,1
1,2006-01-02 08:47:00,6470,6480,6465,6470,589,1.050681e-02,1,1,0.0,1
2,2006-01-02 08:48:00,6472,6473,6461,6465,633,2.101245e-02,1,1,0.0,1
3,2006-01-02 08:49:00,6464,6467,6456,6457,681,3.151578e-02,1,1,0.0,1
4,2006-01-02 08:50:00,6460,6463,6452,6455,385,4.201562e-02,1,1,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...
812095,2016-12-30 13:41:00,9261,9262,9260,9262,442,4.201562e-02,12,5,1.0,5
812096,2016-12-30 13:42:00,9261,9262,9261,9261,275,3.151578e-02,12,5,1.0,5
812097,2016-12-30 13:43:00,9261,9263,9261,9262,672,2.101245e-02,12,5,1.0,5
812098,2016-12-30 13:44:00,9261,9263,9261,9261,646,1.050681e-02,12,5,1.0,5


In [18]:
data["WeekNum"] = data.apply(func.WeekTime, axis=1)
data

,ts,open,high,low,close,vol,IntraDaily_Time,Month,Weekday,Intramonth,IntramonthPeriod,WeekNum
0,2006-01-02 08:46:00,6451,6490,6411,6478,775,0.000000e+00,1,1,0.0,1,0.0
1,2006-01-02 08:47:00,6470,6480,6465,6470,589,1.050681e-02,1,1,0.0,1,0.0
2,2006-01-02 08:48:00,6472,6473,6461,6465,633,2.101245e-02,1,1,0.0,1,0.0
3,2006-01-02 08:49:00,6464,6467,6456,6457,681,3.151578e-02,1,1,0.0,1,0.0
4,2006-01-02 08:50:00,6460,6463,6452,6455,385,4.201562e-02,1,1,0.0,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
812095,2016-12-30 13:41:00,9261,9262,9260,9262,442,4.201562e-02,12,5,1.0,5,1.0
812096,2016-12-30 13:42:00,9261,9262,9261,9261,275,3.151578e-02,12,5,1.0,5,1.0
812097,2016-12-30 13:43:00,9261,9263,9261,9262,672,2.101245e-02,12,5,1.0,5,1.0
812098,2016-12-30 13:44:00,9261,9263,9261,9261,646,1.050681e-02,12,5,1.0,5,1.0


In [23]:
df_list = [group[1] for group in data.groupby(data["ts"].dt.date)]
cum_list = []

if __name__ == '__main__':
    with Pool(processes=12) as pool:
        for i, x in enumerate(tqdm(pool.imap_unordered(func.Cumulative, df_list), total=len(df_list)), 1):
                cum_list.append(x)

data = pd.concat(cum_list)
data = data.sort_values(by="ts").reset_index(drop=True)
data

,ts,open,high,low,close,vol,IntraDaily_Time,Month,Weekday,Intramonth,IntramonthPeriod,WeekNum,cumvol,PreCumVol,PreOpen,PreClose,PreHigh,PreLow
0,2006-01-02 08:46:00,6451,6490,6411,6478,775,0.000000e+00,1,1,0.0,1,0.0,775,160.0,6488.0,6493.0,6493.0,6486.0
1,2006-01-02 08:47:00,6470,6480,6465,6470,589,1.050681e-02,1,1,0.0,1,0.0,1364,257.0,6492.0,6495.0,6497.0,6492.0
2,2006-01-02 08:48:00,6472,6473,6461,6465,633,2.101245e-02,1,1,0.0,1,0.0,1997,345.0,6495.0,6493.0,6496.0,6493.0
3,2006-01-02 08:49:00,6464,6467,6456,6457,681,3.151578e-02,1,1,0.0,1,0.0,2678,480.0,6493.0,6492.0,6494.0,6491.0
4,2006-01-02 08:50:00,6460,6463,6452,6455,385,4.201562e-02,1,1,0.0,1,0.0,3063,591.0,6492.0,6489.0,6492.0,6489.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
812095,2016-12-30 13:41:00,9261,9262,9260,9262,442,4.201562e-02,12,5,1.0,5,1.0,113722,89930.0,9181.0,9181.0,9181.0,9180.0
812096,2016-12-30 13:42:00,9261,9262,9261,9261,275,3.151578e-02,12,5,1.0,5,1.0,113997,90397.0,9180.0,9179.0,9181.0,9179.0
812097,2016-12-30 13:43:00,9261,9263,9261,9262,672,2.101245e-02,12,5,1.0,5,1.0,114669,90814.0,9180.0,9180.0,9180.0,9179.0
812098,2016-12-30 13:44:00,9261,9263,9261,9261,646,1.050681e-02,12,5,1.0,5,1.0,115315,91207.0,9180.0,9180.0,9180.0,9179.0


In [24]:
term = len(timelist)
data["PreCumVol"], data["PreOpen"], data["PreClose"], data["PreHigh"], data["PreLow"] = data["cumvol"].shift(term), data["open"].shift(term), data["close"].shift(term), data["high"].shift(term), data["low"].shift(term)
data

,ts,open,high,low,close,vol,IntraDaily_Time,Month,Weekday,Intramonth,IntramonthPeriod,WeekNum,cumvol,PreCumVol,PreOpen,PreClose,PreHigh,PreLow
0,2006-01-02 08:46:00,6451,6490,6411,6478,775,0.000000e+00,1,1,0.0,1,0.0,775,NaN,NaN,NaN,NaN,NaN
1,2006-01-02 08:47:00,6470,6480,6465,6470,589,1.050681e-02,1,1,0.0,1,0.0,1364,NaN,NaN,NaN,NaN,NaN
2,2006-01-02 08:48:00,6472,6473,6461,6465,633,2.101245e-02,1,1,0.0,1,0.0,1997,NaN,NaN,NaN,NaN,NaN
3,2006-01-02 08:49:00,6464,6467,6456,6457,681,3.151578e-02,1,1,0.0,1,0.0,2678,NaN,NaN,NaN,NaN,NaN
4,2006-01-02 08:50:00,6460,6463,6452,6455,385,4.201562e-02,1,1,0.0,1,0.0,3063,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
812095,2016-12-30 13:41:00,9261,9262,9260,9262,442,4.201562e-02,12,5,1.0,5,1.0,113722,89930.0,9181.0,9181.0,9181.0,9180.0
812096,2016-12-30 13:42:00,9261,9262,9261,9261,275,3.151578e-02,12,5,1.0,5,1.0,113997,90397.0,9180.0,9179.0,9181.0,9179.0
812097,2016-12-30 13:43:00,9261,9263,9261,9262,672,2.101245e-02,12,5,1.0,5,1.0,114669,90814.0,9180.0,9180.0,9180.0,9179.0
812098,2016-12-30 13:44:00,9261,9263,9261,9261,646,1.050681e-02,12,5,1.0,5,1.0,115315,91207.0,9180.0,9180.0,9180.0,9179.0


In [32]:
df_list = [group[1] for group in data.groupby(data["ts"].dt.date)]
OHLCV_list = []

if __name__ == '__main__':
    with Pool(processes=12) as pool:
        for i, x in enumerate(tqdm(pool.imap_unordered(func.DailyOHLCV, df_list), total=len(df_list)), 1):
                OHLCV_list.append(x)

data = pd.concat(OHLCV_list)
data = data.sort_values(by="ts").reset_index(drop=True)
data

,ts,open,high,low,close,vol,IntraDaily_Time,Month,Weekday,Intramonth,...,PreCumVol,PreOpen,PreClose,PreHigh,PreLow,DailyOpen,DailyHigh,DailyLow,DailyClose,DailyVol
0,2006-01-02 08:46:00,6451,6490,6411,6478,775,0.000000e+00,1,1,0.0,...,NaN,NaN,NaN,NaN,NaN,6451,6507,6411,6478,27202
1,2006-01-02 08:47:00,6470,6480,6465,6470,589,1.050681e-02,1,1,0.0,...,NaN,NaN,NaN,NaN,NaN,6451,6507,6411,6478,27202
2,2006-01-02 08:48:00,6472,6473,6461,6465,633,2.101245e-02,1,1,0.0,...,NaN,NaN,NaN,NaN,NaN,6451,6507,6411,6478,27202
3,2006-01-02 08:49:00,6464,6467,6456,6457,681,3.151578e-02,1,1,0.0,...,NaN,NaN,NaN,NaN,NaN,6451,6507,6411,6478,27202
4,2006-01-02 08:50:00,6460,6463,6452,6455,385,4.201562e-02,1,1,0.0,...,NaN,NaN,NaN,NaN,NaN,6451,6507,6411,6478,27202
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
812095,2016-12-30 13:41:00,9261,9262,9260,9262,442,4.201562e-02,12,5,1.0,...,89930.0,9181.0,9181.0,9181.0,9180.0,9215,9282,9215,9262,116363
812096,2016-12-30 13:42:00,9261,9262,9261,9261,275,3.151578e-02,12,5,1.0,...,90397.0,9180.0,9179.0,9181.0,9179.0,9215,9282,9215,9262,116363
812097,2016-12-30 13:43:00,9261,9263,9261,9262,672,2.101245e-02,12,5,1.0,...,90814.0,9180.0,9180.0,9180.0,9179.0,9215,9282,9215,9262,116363
812098,2016-12-30 13:44:00,9261,9263,9261,9261,646,1.050681e-02,12,5,1.0,...,91207.0,9180.0,9180.0,9180.0,9179.0,9215,9282,9215,9262,116363


In [36]:
term = len(timelist)
data["PreDailyVol"], data["PreDailyOpen"], data["PreDailyClose"], data["PreDailyHigh"], data["PreDailyLow"] = data["DailyVol"].shift(term), data["DailyOpen"].shift(term), data["DailyClose"].shift(term), data["DailyHigh"].shift(term), data["DailyLow"].shift(term)
data

,ts,open,high,low,close,vol,IntraDaily_Time,Month,Weekday,Intramonth,...,DailyOpen,DailyHigh,DailyLow,DailyClose,DailyVol,PreDailyVol,PreDailyOpen,PreDailyClose,PreDailyHigh,PreDailyLow
0,2006-01-02 08:46:00,6451,6490,6411,6478,775,0.000000e+00,1,1,0.0,...,6451,6507,6411,6478,27202,NaN,NaN,NaN,NaN,NaN
1,2006-01-02 08:47:00,6470,6480,6465,6470,589,1.050681e-02,1,1,0.0,...,6451,6507,6411,6478,27202,NaN,NaN,NaN,NaN,NaN
2,2006-01-02 08:48:00,6472,6473,6461,6465,633,2.101245e-02,1,1,0.0,...,6451,6507,6411,6478,27202,NaN,NaN,NaN,NaN,NaN
3,2006-01-02 08:49:00,6464,6467,6456,6457,681,3.151578e-02,1,1,0.0,...,6451,6507,6411,6478,27202,NaN,NaN,NaN,NaN,NaN
4,2006-01-02 08:50:00,6460,6463,6452,6455,385,4.201562e-02,1,1,0.0,...,6451,6507,6411,6478,27202,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
812095,2016-12-30 13:41:00,9261,9262,9260,9262,442,4.201562e-02,12,5,1.0,...,9215,9282,9215,9262,116363,92394.0,9188.0,9184.0,9211.0,9168.0
812096,2016-12-30 13:42:00,9261,9262,9261,9261,275,3.151578e-02,12,5,1.0,...,9215,9282,9215,9262,116363,92394.0,9188.0,9184.0,9211.0,9168.0
812097,2016-12-30 13:43:00,9261,9263,9261,9262,672,2.101245e-02,12,5,1.0,...,9215,9282,9215,9262,116363,92394.0,9188.0,9184.0,9211.0,9168.0
812098,2016-12-30 13:44:00,9261,9263,9261,9261,646,1.050681e-02,12,5,1.0,...,9215,9282,9215,9262,116363,92394.0,9188.0,9184.0,9211.0,9168.0


In [38]:
data.to_csv('D:\Strategic_Trading\PriceForecast\data\RTD\RTD_Feature.csv', index=False)